In [ ]:
import time
import json
import math
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from py2neo import Graph, Node, Relationship
%matplotlib inline



In [ ]:
def query_to_df(query, graph):
    print("Starting query...", end=" ")
    query_start_time = time.time()
    df = graph.run(query).to_data_frame()
    print("Done ({:.2f} minutes).".format((time.time()-query_start_time)/60))
    return df

In [ ]:
graph = Graph( "bolt://matlaber10.media.mit.edu:7687", auth=('neo4j','myneo'))
print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
    graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
    graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

#### Community focus

In [ ]:
query = """call apoc.export.csv.query("

// denominator

MATCH (u:NatureAuthor)

MATCH (u)-[e:COAUTHOR]-(b:NatureAuthor)

WITH e.strength AS weight, u, COUNT(b) AS k_u

WITH u, toFloat(sum(weight*k_u))/sum(k_u) AS den



// intracommunity focus

OPTIONAL MATCH (u)-[e:COAUTHOR]-(b:NatureAuthor)

WHERE last(b.louvain) = last(u.louvain)

WITH den, u, COUNT(b) AS k_intra_u, CASE WHEN e IS NULL THEN 0 ELSE e.strength END AS weight

WITH den, u, CASE WHEN sum(k_intra_u)=0 THEN 0 ELSE toFloat(sum(weight*k_intra_u))/sum(k_intra_u) END AS num

WITH den, u, CASE WHEN den=0 THEN 0 ELSE num/den END AS NatureIntraCommunityFocus

WITH den, u, NatureIntraCommunityFocus



// intercommunity focus

OPTIONAL MATCH (u)-[e:COAUTHOR]-(b:NatureAuthor)

WHERE last(b.louvain) <> last(u.louvain)

WITH den, u, NatureIntraCommunityFocus, COUNT(b) AS k_inter_u, CASE WHEN e IS NULL THEN 0 ELSE e.strength END AS weight

WITH den, u, NatureIntraCommunityFocus, CASE WHEN sum(k_inter_u)=0 THEN 0 ELSE toFloat(sum(weight*k_inter_u))/sum(k_inter_u) END AS num

WITH u, NatureIntraCommunityFocus, CASE WHEN den=0 THEN 0 ELSE num/den END AS NatureInterCommunityFocus



RETURN u.name, NatureIntraCommunityFocus, NatureInterCommunityFocus

","/import/result/author_focus.csv",{})"""

df_community_focus = query_to_df(query, graph)

#### Professor connections

In [ ]:
#Augment nodes with pct_last_author
query = """call apoc.periodic.iterate(
"MATCH (a:Top42Author) RETURN a",
"MATCH (a)-[r:AUTHORED]->(:Quanta)
WITH a, toFloat(SUM(CASE WHEN r.is_last_author THEN 1 ELSE 0 END))/COUNT(r) as pct_last_author
SET a.pct_last_author = pct_last_author", {batchSize:100, parallel:true})
"""

df_last_author = query_to_df(query, graph)

In [ ]:
query = """
    MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
    WITH a, COUNT(q) AS num_pubs
    MATCH (a)-[:AUTHORED]-(q:Quanta)-[:AUTHORED]-(b:NatureAuthor)
    WHERE b.pct_last_author > .5
    WITH a, num_pubs, COUNT(distinct q) as num_coauthored_with_prof
    RETURN a.name AS name, num_coauthored_with_prof as num_pubs_with_prof, num_pubs, toFloat(num_coauthored_with_prof)/num_pubs AS ratio_pubs_with_prof
"""
profs_per_paper = query_to_df(query, graph)

In [ ]:
profs_per_paper

In [ ]:
profs_per_paper.to_csv('C:\\Users\\Brend\\Downloads\\paper.csv', index = False, encoding = "UTF-8")

In [ ]:
#Ratio of prof-prof connected papers
#WORKING
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
WITH a, COUNT(q) AS num_pubs
MATCH (a)-[r:COAUTHOR]-(b:NatureAuthor)
WHERE b.pct_last_author > .5
WITH a, num_pubs, SUM(r.strength42) as num_coauthored_with_prof
RETURN a.name AS name, num_coauthored_with_prof as prof_connections, num_pubs, toFloat(num_coauthored_with_prof)/num_pubs AS coauthor_profs_ratio"""

df_coauthor_profs_ratio = query_to_df(query, graph)

In [ ]:
df_coauthor_papers_ratio.sort_values('coauthor_profs_ratio', ascending=False).tail(10)

In [ ]:
df_coauthor_profs_ratio.to_csv('C:\\Users\\Brend\\Downloads\\coauthor_profs_ratio.csv', index = False, encoding = "UTF-8")

#### Cross-cluster edges

In [ ]:
#Ratio of cross-cluster and intra-cluster edges to total edges
#WORKING
query = """
MATCH (u:NatureAuthor)
MATCH (u)-[e:COAUTHOR]-(b:NatureAuthor)
WITH u, COUNT(b) AS k_u
WITH u, k_u AS den

// intracommunity edges
OPTIONAL MATCH (u)-[e:COAUTHOR]-(b:NatureAuthor)
WHERE last(b.louvain) = last(u.louvain)
WITH den, u, COUNT(b) AS k_intra_u
WITH den, u, k_intra_u AS IntraClusterConns, CASE WHEN den=0 THEN 0 ELSE toFloat(k_intra_u)/den END AS IntraClusterRatio
WITH den, u, IntraClusterRatio, IntraClusterConns, 1.0 - IntraClusterRatio AS InterClusterRatio

RETURN u.name AS name, IntraClusterRatio, InterClusterRatio, den AS TotalConns, IntraClusterConns, den - IntraClusterConns AS InterClusterConns
"""

df_clusters = query_to_df(query, graph)

In [ ]:
df_clusters.head(15)

In [ ]:
df_clusters.to_csv('C:\\Users\\Brend\\Downloads\\clusters.csv', index = False, encoding = "UTF-8")

#### Preparing metrics csv

In [ ]:
df_clusters = pd.read_csv('C:\\Users\\Brend\\Downloads\\clusters.csv', index_col = 'name', encoding = "UTF-8")

In [ ]:
df_papers = pd.read_csv('C:\\Users\\Brend\\Downloads\\papers.csv', index_col = 'name', encoding = "UTF-8")

In [ ]:
df_profs = pd.read_csv('C:\\Users\\Brend\\Downloads\\profs.csv', index_col = 'name', encoding = 'UTF-8')

In [ ]:
df_pct_collab = pd.read_csv('C:\\Users\\Brend\\Downloads\\pct_collaborative_pubs.csv', index_col = 'name', encoding = 'UTF-8')

In [ ]:
df_author_focus = pd.read_csv('C:\\Users\\Brend\\Downloads\\author_focus.csv', index_col = 'name', encoding = 'UTF-8')

In [ ]:
df_metrics = df_profs[['num_pubs']].join(df_author_focus, how = 'outer')

In [ ]:
df_metrics = df_metrics.join(df_clusters[['TotalConns', 'InterClusterRatio', 'IntraClusterRatio']], how = 'outer')

In [ ]:
df_metrics = df_metrics.join(df_pct_collab, how = 'outer')

In [ ]:
df_metrics.loc['Marvin Minsky']

In [ ]:
df_metrics.to_csv('C:\\Users\\Brend\\Downloads\\metrics.csv', encoding = "UTF-8")

#### All metrics

In [ ]:
df_metrics = pd.read_csv('C:\\Users\\Brend\\Downloads\\metrics.csv', index_col = 'name', encoding = "UTF-8")

In [ ]:
import random
indices = []
for i in range(10):
    indices.append(random.randrange(0, df_metrics.shape[0]))
df_metrics.iloc[indices]

#### Impact

In [ ]:
#    CALL algo.pageRank(
#     'MATCH (p:Quanta {venue:"Nature"}) RETURN id(p) as id',
#     'MATCH (p1:Quanta {venue:"Nature"})-[:CITES]->(p2:Quanta {venue:"Nature"}) RETURN id(p1) as source, id(p2) as target',
#     {graph:'cypher', iterations:35, write:true, writeProperty:"npr"});

In [ ]:
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
WHERE q.venue = 'Nature'
WITH a, COUNT(q) AS num_pubs, SUM(q.npr) AS tot_rank, MIN(q.npr) AS min_rank, MAX(q.npr) AS max_rank, percentileCont(q.npr, 0.5) AS median_rank 
RETURN a.name AS name, num_pubs, toFloat(tot_rank)/num_pubs AS avg_NPR, min_rank AS min_NPR, max_rank AS max_NPR, median_rank AS median_NPR
"""

df_npr = query_to_df(query, graph)

In [ ]:
df_npr.head(2)

In [ ]:
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
WHERE q.venue = 'Nature'
WITH a, COUNT(q) AS num_pubs, SUM(q.nar) AS tot_rank, MIN(q.nar) AS min_rank, MAX(q.nar) AS max_rank, percentileCont(q.nar, 0.5) AS median_rank 
RETURN a.name AS name, num_pubs, toFloat(tot_rank)/num_pubs AS avg_NAR, min_rank AS min_NAR, max_rank AS max_NAR, median_rank AS median_NAR
"""

df_nar = query_to_df(query, graph)

In [ ]:
df_nar.head(2)

In [ ]:
df_npr.to_csv('C:\\Users\\Brend\\Downloads\\npr.csv', index = False, encoding = "UTF-8")
df_nar.to_csv('C:\\Users\\Brend\\Downloads\\nar.csv', index = False, encoding = "UTF-8")

In [ ]:
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
WHERE q.venue = 'Nature'
WITH a, COUNT(q) AS num_pubs, SUM(q.pageRank_2018) AS tot_rank, MIN(q.pageRank_2018) AS min_rank, MAX(q.pageRank_2018) AS max_rank, percentileCont(q.pageRank_2018, 0.5) AS median_rank 
RETURN a.name AS name, num_pubs, toFloat(tot_rank)/num_pubs AS avg_PR, min_rank AS min_PR, max_rank AS max_PR, median_rank AS median_PR
"""

df_pr = query_to_df(query, graph)

In [ ]:
df_pr.head(2)

In [ ]:
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
WHERE q.venue = 'Nature'
WITH a, COUNT(q) AS num_pubs, SUM(q.articleRank2018) AS tot_rank, MIN(q.articleRank2018) AS min_rank, MAX(q.articleRank2018) AS max_rank, percentileCont(q.articleRank2018, 0.5) AS median_rank 
RETURN a.name AS name, num_pubs, toFloat(tot_rank)/num_pubs AS avg_AR, min_rank AS min_AR, max_rank AS max_AR, median_rank AS median_AR
"""

df_ar = query_to_df(query, graph)

In [ ]:
df_ar.head(2)

In [ ]:
df_pr.to_csv('C:\\Users\\Brend\\Downloads\\pr.csv', index = False, encoding = "UTF-8")
df_ar.to_csv('C:\\Users\\Brend\\Downloads\\ar.csv', index = False, encoding = "UTF-8")

In [ ]:
df_npr = pd.read_csv('C:\\Users\\Brend\\Downloads\\npr.csv', index_col = 'name', encoding = "UTF-8")
df_nar = pd.read_csv('C:\\Users\\Brend\\Downloads\\nar.csv', index_col = 'name', encoding = "UTF-8")
df_pr = pd.read_csv('C:\\Users\\Brend\\Downloads\\pr.csv', index_col = 'name', encoding = "UTF-8")
df_ar = pd.read_csv('C:\\Users\\Brend\\Downloads\\ar.csv', index_col = 'name', encoding = "UTF-8")

In [ ]:
df_npr = df_npr[[col for col in df_npr.columns if col != 'num_pubs']]
df_nar = df_nar[[col for col in df_nar.columns if col != 'num_pubs']]
df_pr = df_pr[[col for col in df_pr.columns if col != 'num_pubs']]
df_ar = df_ar[[col for col in df_ar.columns if col != 'num_pubs']]

In [ ]:
df_imp = df_npr.join(df_nar, how = 'outer')
df_imp = df_imp.join(df_pr, how = 'outer')
df_imp = df_imp.join(df_ar, how = 'outer')

In [ ]:
df_imp.to_csv('C:\\Users\\Brend\\Downloads\\impact.csv', encoding = "UTF-8")

#### Collaboration vs Impact

In [ ]:
df_collab = pd.read_csv('C:\\Users\\Brend\\Downloads\\metrics.csv', index_col = 'name', encoding = "UTF-8")
df_impact = pd.read_csv('C:\\Users\\Brend\\Downloads\\impact.csv', index_col = 'name', encoding = "UTF-8")
df_all = df_collab.join(df_impact, how = 'outer')

In [ ]:
df_all.loc

In [ ]:
df_all.to_csv('C:\\Users\\Brend\\Downloads\\correlation.csv', encoding = "UTF-8")

In [ ]:
df_all[df_all.index == 'Eric S. Lander']

In [ ]:
df_plot = df_all[['profs_per_pub', 'median_AR']]
df_plot = df_plot[df_plot['median_AR'] < 50]
df_plot = df_plot[df_plot['median_AR'] > .16]
df_plot = df_plot[df_plot['profs_per_pub'] > .1]

In [ ]:
import seaborn as sns
sns.set()
sns.relplot(x="profs_per_pub", y="median_AR",  
            data=df_plot);

#### Deweighted pct_collaborative_pubs

In [ ]:
# Set num_profs property for Eric Lander's publications
query = """
call apoc.periodic.iterate(
    "MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
	WHERE a.name = 'Eric S. Lander'
    WITH a, COUNT(q) AS num_pubs, COLLECT(q) AS pubs
    UNWIND pubs as pub
    MATCH (b:NatureAuthor)-[:AUTHORED]-(pub)
    WHERE b.pct_last_author > .25
    RETURN pub, COUNT(DISTINCT b.name) as num_profs",
   " SET pub.num_profs = num_profs",
 {batchSize:100, parallel:true})
"""

In [ ]:
# Set num_profs property for all publications
query = """call apoc.periodic.iterate(
    "MATCH (b:NatureAuthor)-[:AUTHORED]-(pub:Quanta)
    WHERE b.pct_last_author > .25
    RETURN pub, COUNT(DISTINCT b) as num_profs",
    "SET pub.num_profs = num_profs", {batchSize:1000, parallel:true})
    """

In [ ]:
# Set num_authors property for all publications
query = """call apoc.periodic.iterate(
    "MATCH (b:NatureAuthor)-[:AUTHORED]-(pub:Quanta)
    RETURN pub, COUNT(DISTINCT b) as num_authors",
    "SET pub.num_authors = num_authors", {batchSize:1000, parallel:true})
    """

Eric Lander metrics

In [ ]:
# Linear-prof deweight for pct_collaborative_pubs for Eric Lander
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
	WHERE a.name = 'Eric S. Lander' AND  q.profs > 1
    WITH a, SUM(toFloat(1)/(q.num_profs - 1)) AS collab_linprof
    MATCH (a)-[:AUTHORED]-(q:Quanta)
    WITH a, COUNT(q) AS num_pubs, collab_linprof
    RETURN a.name, collab_linprof/num_pubs AS pct_collab_linprof
    """
df = query_to_df(query, graph)
df

In [ ]:
# Sqrt-prof deweight for pct_collaborative_pubs for Eric Lander
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
	WHERE a.name = 'Eric S. Lander' AND  q.num_profs > 1
    WITH a, SUM(toFloat(1)/(sqrt(q.num_profs - 1))) AS collab_sqrtprof
    MATCH (a)-[:AUTHORED]-(q:Quanta)
    WITH a, COUNT(q) AS num_pubs, collab_sqrtprof
    RETURN a.name, collab_sqrtprof/num_pubs AS pct_collab_sqrtprof
    """
df = query_to_df(query, graph)
df

In [ ]:
# No deweight for pct_collaborative_pubs for Eric Lander
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
	WHERE a.name = 'Eric S. Lander' AND  q.num_profs > 1
    WITH a, SUM(toFloat(1)/(q.num_profs + 1 - q.num_profs)) AS collab_unweighted
    MATCH (a)-[:AUTHORED]-(q:Quanta)
    WITH a, COUNT(q) AS num_pubs, collab_unweighted
    RETURN a.name, collab_unweighted/num_pubs AS pct_collab_unweighted
    """
df = query_to_df(query, graph)
df

In [ ]:
# Sqrt-author deweight for pct_collaborative_pubs for Eric Lander
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
	WHERE a.name = 'Eric S. Lander' AND  q.num_profs > 1
    WITH a, SUM(toFloat(1)/(sqrt(q.num_authors - 1))) AS collab_sqrtauth
    MATCH (a)-[:AUTHORED]-(q:Quanta)
    WITH a, COUNT(q) AS num_pubs, collab_sqrtauth
    RETURN a.name, collab_sqrtauth/num_pubs AS pct_collab_sqrtauth
    """
df = query_to_df(query, graph)
df

In [ ]:
# Linear-author deweight for pct_collaborative_pubs for Eric Lander
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
	WHERE a.name = 'Eric S. Lander' AND  q.profs > 1
    WITH a, SUM(toFloat(1)/(q.num_authors - 1)) AS collab_linauth
    MATCH (a)-[:AUTHORED]-(q:Quanta)
    WITH a, COUNT(q) AS num_pubs, collab_linauth
    RETURN a.name, collab_linauth/num_pubs AS pct_collab_linauth
    """
df = query_to_df(query, graph)
df

In [ ]:
# All versions of pct_collaborative_pubs for Eric Lander
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
	WHERE a.name = 'Eric S. Lander' AND q.num_profs > 1
    WITH a, 
        SUM(toFloat(1)/(q.num_profs - 1)) AS collab_linprof,
        SUM(toFloat(1)/(sqrt(q.num_profs - 1))) AS collab_sqrtprof,
        SUM(toFloat(1)/(sqrt(q.num_authors - 1))) AS collab_sqrtauth,
        SUM(toFloat(1)/(q.num_authors - 1)) AS collab_linauth,
        SUM(toFloat(1)/(q.num_profs + 1 - q.num_profs)) AS collab_unweighted
    MATCH (a)-[:AUTHORED]-(q:Quanta)
    WITH a, COUNT(q) AS num_pubs, collab_linprof, collab_sqrtauth, collab_linauth, collab_unweighted, collab_sqrtprof
    RETURN a.name, 
        collab_linprof/num_pubs AS pct_collab_linprof,
        collab_sqrtprof/num_pubs AS pct_collab_sqrtprof,
        collab_unweighted/num_pubs AS pct_collab_unweighted,
        collab_sqrtauth/num_pubs AS pct_collab_sqrtauth,
        collab_linauth/num_pubs AS pct_collab_linauth
    """
df = query_to_df(query, graph)
df

All authors

In [ ]:
# All versions of pct_collaborative_pubs for Eric Lander
query = """
MATCH (a:NatureAuthor)-[:AUTHORED]->(q:Quanta)
	WHERE q.num_profs > 1 AND q.num_authors > 1
    WITH a, 
        SUM(toFloat(1)/(q.num_profs - 1)) AS collab_linprof,
        SUM(toFloat(1)/(sqrt(q.num_profs - 1))) AS collab_sqrtprof,
        SUM(toFloat(1)/(sqrt(q.num_authors - 1))) AS collab_sqrtauth,
        SUM(toFloat(1)/(q.num_authors - 1)) AS collab_linauth,
        SUM(toFloat(1)/(q.num_profs + 1 - q.num_profs)) AS collab_unweighted
    MATCH (a)-[:AUTHORED]-(q:Quanta)
    WITH a, COUNT(q) AS num_pubs, collab_linprof, collab_sqrtauth, collab_linauth, collab_unweighted, collab_sqrtprof
    RETURN a.name, 
        collab_linprof/num_pubs AS pct_collab_linprof,
        collab_sqrtprof/num_pubs AS pct_collab_sqrtprof,
        collab_unweighted/num_pubs AS pct_collab_unweighted,
        collab_sqrtauth/num_pubs AS pct_collab_sqrtauth,
        collab_linauth/num_pubs AS pct_collab_linauth
    """
df = query_to_df(query, graph)
df

In [ ]:
df.to_csv('C:\\Users\\Brend\\Downloads\\pct_collaborative_pubs.csv', index = False, encoding = "UTF-8")


In [ ]:
df_pct_collaborative_pubs = pd.read_csv('C:\\Users\\Brend\\Downloads\\pct_collaborative_pubs.csv', index_col = 'name', encoding = 'UTF-8')

In [ ]:
df_percentile = df_pct_collaborative_pubs.copy()
for col in df_pct_collaborative_pubs.columns:
    percentile_col = df_pct_collaborative_pubs[pd.notna(df_pct_collaborative_pubs[col])][col]
    df_percentile[col+'_%'] = percentile_col.rank(pct=True)
df_percentile.loc[['Eric S. Lander', 'Robert Langer', 'Marvin Minsky']][[col for col in df_percentile.columns if '%' in col]]